# MRI-QC
https://mriqc.readthedocs.io/en/latest/docker.html#docker

## Test

In [ ]:
docker run -it nipreps/mriqc:latest --version

## Run the participant level in subjects 001 002 003

If the argument `--participant_label` is not provided, then all subjects will be processed and the group level analysis will automatically be executed without need of running the command in item 3.

Paths `<bids_dir>` and `<output_dir>` must be absolute. In particular, specifying relative paths for `<output_dir>` will generate no error and mriqc will run to completion without error but produce no output.

For security reasons, we recommend to run the docker command with the options `--read-only --tmpfs /run --tmpfs /tmp`. This will run the docker image in read-only mode, and map the temporary folders `/run` and `/tmp` to the temporal folder of the host.

In [ ]:
# default
docker run -it --rm -v <bids_dir>:/data:ro -v <output_dir>:/out nipreps/mriqc:latest /data /out participant --participant_label 001 002 003
# with my paths
docker run -it --rm -v /home/jaimebarranco/Desktop/samples_v3_bids:/data:ro -v /home/jaimebarranco/Desktop/mriqc_output:/out nipreps/mriqc:latest /data /out participant --participant_label 001 002 003
# handle performance
docker run -it --rm -v /home/jaimebarranco/Desktop/samples_v3_bids:/data:ro -v /home/jaimebarranco/Desktop/mriqc_output:/out nipreps/mriqc:latest /data /out participant --participant_label 001 --nprocs 12 --omp-nthreads 12

## Run the group level and report generation on previously processed (use the same `<output_dir>`) subjects

In [ ]:
# default
docker run -it --rm -v <bids_dir>:/data:ro -v <output_dir>:/out nipreps/mriqc:latest /data /out group #--read-only --tmpfs /run --tmpfs /tmp
# with my paths
docker run -it --rm -v /home/jaimebarranco/Desktop/samples_v2_bids:/data:ro -v /home/jaimebarranco/Desktop/mriqc_output:/out nipreps/mriqc:latest /data /out group #--read-only --tmpfs /run --tmpfs /tmp

## Loop of individual participants
And group report

In [ ]:
import os, subprocess

# def run_command(command):
#     process = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
#     console_output, console_errors = process.communicate()

def run_command(command):
    os.system(command)

input_folder = '/home/jaimebarranco/Desktop/samples_v3_bids'
output_folder = '/home/jaimebarranco/Desktop/mriqc_output'
n_procs = 12
n_threads = 12

n_sub = [f.path for f in os.scandir(input_folder) if f.is_dir() and f.name.startswith('sub-')] # number of subjects in input folder
n_sub = len(n_sub)
print(f'Number of subjects: {n_sub}')

# Run MRIQC for each subject
for sub in range(6, n_sub):
    command = f'docker run --rm -v {input_folder}:/data:ro -v {output_folder}:/out nipreps/mriqc:latest /data /out participant --participant_label {sub+1:03} --nprocs {n_procs} --omp-nthreads {n_threads}'
    print(command)
    run_command(command)

# Run MRIQC group analysis
command = f'docker run --rm -v {input_folder}:/data:ro -v {output_folder}:/out nipreps/mriqc:latest /data /out group'
print(command)
run_command(command)

## Metrics correlation
Meri's subjective scores vs IQMs (Image Quality Metrics)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# read an excel file in .xlsx format
df = pd.read_excel('/home/jaimebarranco/Desktop/scores.xlsx', sheet_name='samples_v3')

# metrics
meri = df['quality'].to_numpy()
cjv = df['cjv'].to_numpy()
cnr = df['cnr'].to_numpy()
efc = df['efc'].to_numpy()
fber = df['fber'].to_numpy()
fwhm_avg = df['fwhm_avg'].to_numpy()
fwhm_x = df['fwhm_x'].to_numpy()
fwhm_y = df['fwhm_y'].to_numpy()
fwhm_z = df['fwhm_z'].to_numpy()
icvs_csf = df['icvs_csf'].to_numpy()
icvs_gm = df['icvs_gm'].to_numpy()
icvs_wm = df['icvs_wm'].to_numpy()
inu_med = df['inu_med'].to_numpy()
inu_range = df['inu_range'].to_numpy()
qi_1 = df['qi_1'].to_numpy()
qi_2 = df['qi_2'].to_numpy()
rpve_csf = df['rpve_csf'].to_numpy()
rpve_gm = df['rpve_gm'].to_numpy()
rpve_wm = df['rpve_wm'].to_numpy()
snr_csf = df['snr_csf'].to_numpy()
snr_gm = df['snr_gm'].to_numpy()
snr_total = df['snr_total'].to_numpy()
snr_wm = df['snr_wm'].to_numpy()
snrd_csf = df['snrd_csf'].to_numpy()
snrd_gm = df['snrd_gm'].to_numpy()
snrd_total = df['snrd_total'].to_numpy()
snrd_wm = df['snrd_wm'].to_numpy()
summary_bg_k = df['summary_bg_k'].to_numpy()
summary_bg_mad = df['summary_bg_mad'].to_numpy()
summary_bg_mean = df['summary_bg_mean'].to_numpy()
summary_bg_median = df['summary_bg_median'].to_numpy()
summary_bg_n = df['summary_bg_n'].to_numpy()
summary_bg_p05 = df['summary_bg_p05'].to_numpy()
summary_bg_p95 = df['summary_bg_p95'].to_numpy()
summary_bg_stdv = df['summary_bg_stdv'].to_numpy()
summary_csf_k = df['summary_csf_k'].to_numpy()
summary_csf_mad = df['summary_csf_mad'].to_numpy()
summary_csf_mean = df['summary_csf_mean'].to_numpy()
summary_csf_median = df['summary_csf_median'].to_numpy()
summary_csf_n = df['summary_csf_n'].to_numpy()
summary_csf_p05 = df['summary_csf_p05'].to_numpy()
summary_csf_p95 = df['summary_csf_p95'].to_numpy()
summary_csf_stdv = df['summary_csf_stdv'].to_numpy()
summary_gm_k = df['summary_gm_k'].to_numpy()
summary_gm_mad = df['summary_gm_mad'].to_numpy()
summary_gm_mean = df['summary_gm_mean'].to_numpy()
summary_gm_median = df['summary_gm_median'].to_numpy()
summary_gm_n = df['summary_gm_n'].to_numpy()
summary_gm_p05 = df['summary_gm_p05'].to_numpy()
summary_gm_p95 = df['summary_gm_p95'].to_numpy()
summary_gm_stdv = df['summary_gm_stdv'].to_numpy()
summary_wm_k = df['summary_wm_k'].to_numpy()
summary_wm_mad = df['summary_wm_mad'].to_numpy()
summary_wm_mean = df['summary_wm_mean'].to_numpy()
summary_wm_median = df['summary_wm_median'].to_numpy()
summary_wm_n = df['summary_wm_n'].to_numpy()
summary_wm_p05 = df['summary_wm_p05'].to_numpy()
summary_wm_p95 = df['summary_wm_p95'].to_numpy()
summary_wm_stdv = df['summary_wm_stdv'].to_numpy()
tpm_overlap_csf = df['tpm_overlap_csf'].to_numpy()
tpm_overlap_gm = df['tpm_overlap_gm'].to_numpy()
tpm_overlap_wm = df['tpm_overlap_wm'].to_numpy()
wm2max = df['wm2max'].to_numpy()

# add all the metrics above to a list
metrics = [meri, cjv, cnr, efc, fber, fwhm_avg, fwhm_x, fwhm_y, fwhm_z, icvs_csf, icvs_gm, icvs_wm, inu_med, inu_range, qi_1, qi_2, rpve_csf, rpve_gm, rpve_wm, snr_csf, snr_gm, snr_total, snr_wm, snrd_csf, snrd_gm, snrd_total, snrd_wm, summary_bg_k, summary_bg_mad, summary_bg_mean, summary_bg_median, summary_bg_n, summary_bg_p05, summary_bg_p95, summary_bg_stdv, summary_csf_k, summary_csf_mad, summary_csf_mean, summary_csf_median, summary_csf_n, summary_csf_p05, summary_csf_p95, summary_csf_stdv, summary_gm_k, summary_gm_mad, summary_gm_mean, summary_gm_median, summary_gm_n, summary_gm_p05, summary_gm_p95, summary_gm_stdv, summary_wm_k, summary_wm_mad, summary_wm_mean, summary_wm_median, summary_wm_n, summary_wm_p05, summary_wm_p95, summary_wm_stdv, tpm_overlap_csf, tpm_overlap_gm, tpm_overlap_wm, wm2max]
metrics_name = [i for i in df.columns if i not in ['subject', 'sub number', 'comments', 'Sex', 'Age', 'Height', 'Weight', 'BMI', 'axial_length']]

ALPHA = 0.05

# correlation between Meri and all the metrics
for i in range(len(metrics)):
    # R-squared measures the proportion of the variance in one variable that is predictable from the other variable.
    # p_value is a measure of the evidence against a null hypothesis. A small p-value indicates strong evidence against the null hypothesis, while a large p-value indicates weak evidence against the null hypothesis 
    r_squared, p_value = stats.pearsonr(meri, metrics[i])
    if p_value < ALPHA:
        print(f'{metrics_name[i]}: r^2 = {r_squared:.4}, p-value = {p_value:.4}')

# plot Meri vs. all the metrics
# for i in range(len(metrics)):
#     plt.figure(figsize=(10, 10))
#     plt.scatter(meri, metrics[i])
#     plt.xlabel('Meri')
#     plt.ylabel(metrics_name[i])
#     plt.show()


## 95% confidence interval: qi_2, summary_bg_mad, summary_gm_mean

In [ ]:
# subjects outside the 95% confidence interval for qi_2
relevant_metrics = [qi_2, summary_bg_mad, summary_gm_mean]
relevant_metrics_name = ['qi_2', 'summary_bg_mad', 'summary_gm_mean']

for i in range(len(relevant_metrics)):
    print(f'\n{relevant_metrics_name[i]}')
    mean = np.mean(relevant_metrics[i])
    std = np.std(relevant_metrics[i])
    lower = mean - 2*std
    upper = mean + 2*std
    print(f'Lower: {lower}, Upper: {upper}')
    for j in range(len(relevant_metrics[i])):
        if relevant_metrics[i][j] < lower or relevant_metrics[i][j] > upper:
            print(f'{j+1:03} - {relevant_metrics[i][j]}')

# Fetal Brain QC (Thomas)

In [ ]:
# usage: qc_list_bids_csv [-h] [--mask-patterns MASK_PATTERNS [MASK_PATTERNS ...]] [--out-csv OUT_CSV] [--anonymize-name | --no-anonymize-name] bids-dir
qc_list_bids_csv --mask-patterns-base /home/jaimebarranco/Desktop/samples_v3_bids/derivatives/masks/ --mask-patterns "sub-{subject}_mask.nii.gz" --out-csv /home/jaimebarranco/Desktop/fetal_qc_output/bids_csv.csv /home/jaimebarranco/Desktop/samples_v3_bids
qc_list_bids_csv --mask-patterns-base /home/jaimebarranco/Desktop/samples_v3_bids/derivatives/masks/ --mask-patterns "sub-{subject}_mask.nii.gz" --out-csv /home/jaimebarranco/Desktop/fetal_qc_output/bids_csv.csv /home/jaimebarranco/Desktop/samples_v3_bids

In [ ]:
qc_run_pipeline --bids_dir /home/jaimebarranco/Desktop/samples_v3_bids --out_path /home/jaimebarranco/Desktop/fetal_qc_output/ --mask-patterns-base /home/jaimebarranco/Desktop/samples_v3_bids/derivatives/masks/ --mask-patterns "_mask.nii.gz"
qc_run_pipeline --bids_dir /home/jaimebarranco/Desktop/samples_v3_bids --out_path /home/jaimebarranco/Desktop/fetal_qc_output/

In [ ]:
# Thomas commands - branch mreye
# being in the data folder
qc_list_bids_csv . --mask-patterns-base derivatives/masks/ --out-csv bids.csv --mask-patterns "sub-{subject}_mask.nii.gz" --suffix T1w # index the folder
qc_generate_reports derivatives/reports bids.csv # generate the reports
# mine
qc_list_bids_csv /home/jaimebarranco/Desktop/samples_v3_bids --mask-patterns-base /home/jaimebarranco/Desktop/samples_v3_bids/derivatives/masks/ --mask-patterns "sub-{subject}_mask.nii.gz" --suffix T1w --out-csv /home/jaimebarranco/Desktop/fetal_qc_output/bids_csv.csv
qc_generate_reports /home/jaimebarranco/Desktop/fetal_qc_output/ /home/jaimebarranco/Desktop/fetal_qc_output/bids_csv.csv # outpath bids.csv
qc_generate_index /home/jaimebarranco/Desktop/fetal_qc_output/ # generate index.html
# test
qc_list_bids_csv /home/jaimebarranco/Desktop/samples_v3_bids_test --mask-patterns-base /home/jaimebarranco/Desktop/samples_v3_bids_test/derivatives/masks/ --mask-patterns "sub-{subject}_mask.nii.gz" --suffix T1w --out-csv /home/jaimebarranco/Desktop/fetal_qc_output_test/bids_csv.csv
qc_generate_reports /home/jaimebarranco/Desktop/fetal_qc_output_test/ /home/jaimebarranco/Desktop/fetal_qc_output_test/bids_csv.csv # outpath bids.csv
# pipeline
qc_run_pipeline --bids_dir /home/jaimebarranco/Desktop/samples_v3_bids --out_path /home/jaimebarranco/Desktop/fetal_qc_output/ --mask-patterns-base /home/jaimebarranco/Desktop/samples_v3_bids/derivatives/masks/ --mask-patterns "sub-{subject}_mask.nii.gz" --suffix T1w

# MREye-QC

In [ ]:
# mreye-qc
docker run -it --rm -v /home/jaimebarranco/Desktop/samples_v3_bids:/data:ro -v /home/jaimebarranco/Desktop/mreyeqc_output:/out mreyeqc:latest /data /out participant --participant_label 001 --nprocs 12 --omp-nthreads 12
# 7bd99c588704826d399898ec1f7419d40dc09b27473d511066f3cf8ab097fe5a

Loop

In [ ]:
import os, subprocess

# def run_command(command):
#     process = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
#     console_output, console_errors = process.communicate()

def run_command(command):
    os.system(command)

input_folder = '/home/jaimebarranco/Desktop/samples_v3_bids'
output_folder = '/home/jaimebarranco/Desktop/mreyeqc_output'
n_procs = 12
n_threads = 12

n_sub = [f.path for f in os.scandir(input_folder) if f.is_dir() and f.name.startswith('sub-')] # number of subjects in input folder
n_sub = len(n_sub)
print(f'Number of subjects: {n_sub}')

# Run MRIQC for each subject
for sub in range(1, 6):
    command = f'docker run --rm -v {input_folder}:/data:ro -v {output_folder}:/out mreyeqc_test:latest /data /out participant --participant_label {sub+1:03} --nprocs {n_procs} --omp-nthreads {n_threads}'
    print(command)
    run_command(command)


In [ ]:
# Run MRIQC group analysis
command = f'docker run --rm -v {input_folder}:/data:ro -v {output_folder}:/out mreyeqc_test:latest /data /out group'
print(command)
run_command(command)

## Metrics correlation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# read an excel file in .xlsx format
df = pd.read_excel('/home/jaimebarranco/Desktop/scores.xlsx', sheet_name='mriqc_eye_mask')

# metrics
meri = df['quality'].to_numpy()
cjv = df['cjv'].to_numpy()
cnr = df['cnr'].to_numpy()
efc = df['efc'].to_numpy()
fber = df['fber'].to_numpy()
fwhm_avg = df['fwhm_avg'].to_numpy()
fwhm_x = df['fwhm_x'].to_numpy()
fwhm_y = df['fwhm_y'].to_numpy()
fwhm_z = df['fwhm_z'].to_numpy()
icvs_csf = df['icvs_csf'].to_numpy()
icvs_gm = df['icvs_gm'].to_numpy()
icvs_wm = df['icvs_wm'].to_numpy()
inu_med = df['inu_med'].to_numpy()
inu_range = df['inu_range'].to_numpy()
qi_1 = df['qi_1'].to_numpy()
qi_2 = df['qi_2'].to_numpy()
rpve_csf = df['rpve_csf'].to_numpy()
rpve_gm = df['rpve_gm'].to_numpy()
rpve_wm = df['rpve_wm'].to_numpy()
snr_csf = df['snr_csf'].to_numpy()
snr_gm = df['snr_gm'].to_numpy()
snr_total = df['snr_total'].to_numpy()
snr_wm = df['snr_wm'].to_numpy()
snrd_csf = df['snrd_csf'].to_numpy()
snrd_gm = df['snrd_gm'].to_numpy()
snrd_total = df['snrd_total'].to_numpy()
snrd_wm = df['snrd_wm'].to_numpy()
summary_bg_k = df['summary_bg_k'].to_numpy()
summary_bg_mad = df['summary_bg_mad'].to_numpy()
summary_bg_mean = df['summary_bg_mean'].to_numpy()
summary_bg_median = df['summary_bg_median'].to_numpy()
summary_bg_n = df['summary_bg_n'].to_numpy()
summary_bg_p05 = df['summary_bg_p05'].to_numpy()
summary_bg_p95 = df['summary_bg_p95'].to_numpy()
summary_bg_stdv = df['summary_bg_stdv'].to_numpy()
summary_csf_k = df['summary_csf_k'].to_numpy()
summary_csf_mad = df['summary_csf_mad'].to_numpy()
summary_csf_mean = df['summary_csf_mean'].to_numpy()
summary_csf_median = df['summary_csf_median'].to_numpy()
summary_csf_n = df['summary_csf_n'].to_numpy()
summary_csf_p05 = df['summary_csf_p05'].to_numpy()
summary_csf_p95 = df['summary_csf_p95'].to_numpy()
summary_csf_stdv = df['summary_csf_stdv'].to_numpy()
summary_gm_k = df['summary_gm_k'].to_numpy()
summary_gm_mad = df['summary_gm_mad'].to_numpy()
summary_gm_mean = df['summary_gm_mean'].to_numpy()
summary_gm_median = df['summary_gm_median'].to_numpy()
summary_gm_n = df['summary_gm_n'].to_numpy()
summary_gm_p05 = df['summary_gm_p05'].to_numpy()
summary_gm_p95 = df['summary_gm_p95'].to_numpy()
summary_gm_stdv = df['summary_gm_stdv'].to_numpy()
summary_wm_k = df['summary_wm_k'].to_numpy()
summary_wm_mad = df['summary_wm_mad'].to_numpy()
summary_wm_mean = df['summary_wm_mean'].to_numpy()
summary_wm_median = df['summary_wm_median'].to_numpy()
summary_wm_n = df['summary_wm_n'].to_numpy()
summary_wm_p05 = df['summary_wm_p05'].to_numpy()
summary_wm_p95 = df['summary_wm_p95'].to_numpy()
summary_wm_stdv = df['summary_wm_stdv'].to_numpy()
tpm_overlap_csf = df['tpm_overlap_csf'].to_numpy()
tpm_overlap_gm = df['tpm_overlap_gm'].to_numpy()
tpm_overlap_wm = df['tpm_overlap_wm'].to_numpy()
wm2max = df['wm2max'].to_numpy()

# add all the metrics above to a list
metrics = [meri, cjv, cnr, efc, fber, fwhm_avg, fwhm_x, fwhm_y, fwhm_z, icvs_csf, icvs_gm, icvs_wm, inu_med, inu_range, qi_1, qi_2, rpve_csf, rpve_gm, rpve_wm, snr_csf, snr_gm, snr_total, snr_wm, snrd_csf, snrd_gm, snrd_total, snrd_wm, summary_bg_k, summary_bg_mad, summary_bg_mean, summary_bg_median, summary_bg_n, summary_bg_p05, summary_bg_p95, summary_bg_stdv, summary_csf_k, summary_csf_mad, summary_csf_mean, summary_csf_median, summary_csf_n, summary_csf_p05, summary_csf_p95, summary_csf_stdv, summary_gm_k, summary_gm_mad, summary_gm_mean, summary_gm_median, summary_gm_n, summary_gm_p05, summary_gm_p95, summary_gm_stdv, summary_wm_k, summary_wm_mad, summary_wm_mean, summary_wm_median, summary_wm_n, summary_wm_p05, summary_wm_p95, summary_wm_stdv, tpm_overlap_csf, tpm_overlap_gm, tpm_overlap_wm, wm2max]
metrics_name = [i for i in df.columns if i not in ['subject', 'sub number', 'comments', 'Sex', 'Age', 'Height', 'Weight', 'BMI', 'axial_length']]

ALPHA = 0.05

# correlation between Meri and all the metrics
for i in range(len(metrics)):
    # R-squared measures the proportion of the variance in one variable that is predictable from the other variable.
    # p_value is a measure of the evidence against a null hypothesis. A small p-value indicates strong evidence against the null hypothesis, while a large p-value indicates weak evidence against the null hypothesis 
    r_squared, p_value = stats.pearsonr(meri, metrics[i])
    if p_value < ALPHA:
        print(f'{metrics_name[i]}: r^2 = {r_squared:.4}, p-value = {p_value:.4}')

# plot Meri vs. all the metrics
# for i in range(len(metrics)):
#     plt.figure(figsize=(10, 10))
#     plt.scatter(meri, metrics[i])
#     plt.xlabel('Meri')
#     plt.ylabel(metrics_name[i])
#     plt.show()


## 95% confidence interval: snr_csf, summary_csf_mean, summary_wm_mad

In [ ]:
# subjects outside the 95% confidence interval for qi_2
relevant_metrics = [snr_csf, summary_csf_mean, summary_wm_mad]
relevant_metrics_name = ['snr_csf', 'summary_csf_mean', 'summary_wm_mad']

for i in range(len(relevant_metrics)):
    print(f'\n{relevant_metrics_name[i]}')
    mean = np.mean(relevant_metrics[i])
    std = np.std(relevant_metrics[i])
    lower = mean - 2*std
    upper = mean + 2*std
    print(f'Lower: {lower}, Upper: {upper}')
    for j in range(len(relevant_metrics[i])):
        if relevant_metrics[i][j] < lower or relevant_metrics[i][j] > upper:
            print(f'{j+1:03} - {relevant_metrics[i][j]}')

## Correlation Meri - Bene

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# read an excel file in .xlsx format
df_meri = pd.read_csv('/home/jaimebarranco/Desktop/SHIP_QCmri_MBC/ratings.csv')
df_bene = pd.read_csv('/home/jaimebarranco/Desktop/Evaluations_frb/ratings.csv')

# metrics meri
rating_meri = df_meri['rating'].to_numpy()
blur_meri = df_meri['blur'].to_numpy()
noise_meri = df_meri['noise'].to_numpy()
motion_meri = df_meri['motion'].to_numpy()
bgair_meri = df_meri['bgair'].to_numpy()
eyes_closed_meri = np.where(df_meri['artifacts'].to_numpy() == "['eyes-closed']", 1, 0)
# selected_slices_meri = df_meri['selected_slices'].to_numpy()

# metrics bene
rating_bene = df_bene['rating'].to_numpy()
blur_bene = df_bene['blur'].to_numpy()
noise_bene = df_bene['noise'].to_numpy()
motion_bene = df_bene['motion'].to_numpy()
bgair_bene = df_bene['bgair'].to_numpy()
eyes_closed_bene = np.where(df_bene['artifacts'].to_numpy() == "['eyes-closed']", 1, 0)
# selected_slices_bene = df_bene['selected_slices'].to_numpy()


# add all the metrics above to a list
metrics_meri = [rating_meri, blur_meri, noise_meri, motion_meri, bgair_meri, eyes_closed_meri] #, selected_slices_meri]
metrics_bene = [rating_bene, blur_bene, noise_bene, motion_bene, bgair_bene, eyes_closed_bene] #, selected_slices_bene]
metrics_names = ['rating', 'blur', 'noise', 'motion', 'bgair', 'eyes_closed'] #, 'selected_slices']

ALPHA = 0.05

# correlation between Meri and Bene
for i in range(len(metrics_meri)):
    # R-squared measures the proportion of the variance in one variable that is predictable from the other variable.
    # p_value is a measure of the evidence against a null hypothesis. A small p-value indicates strong evidence against the null hypothesis, while a large p-value indicates weak evidence against the null hypothesis 
    r_squared, p_value = stats.pearsonr(metrics_meri[i], metrics_bene[i])
    # if p_value < ALPHA:
    print(f'{metrics_names[i]}: r^2 = {r_squared:.4}, p-value = {p_value:.4}')

# Eyes closed correlation
sum = 0
for i in range(len(eyes_closed_meri)):
    if eyes_closed_meri[i] == eyes_closed_bene[i]: sum += 1 
print(f'eyes_closed: {np.round(sum/len(eyes_closed_meri)*100, 2)}%')

rating: r^2 = 0.7436, p-value = 8.139e-16
blur: r^2 = 0.5103, p-value = 8.28e-07
noise: r^2 = 0.52, p-value = 4.699e-07
motion: r^2 = 0.4279, p-value = 5.464e-05
bgair: r^2 = 0.5281, p-value = 2.892e-07
eyes_closed: r^2 = 0.07868, p-value = 0.4795
eyes_closed: 66.27%


## Brainmask - Meri

Metrics correlation with Meri's ratings

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# read an excel file in .xlsx format
df = pd.read_excel('/home/jaimebarranco/Desktop/scores.xlsx', sheet_name='mreyeqc_brainmask_meri')

# metrics
meri = df['rating'].to_numpy()
cjv = df['cjv'].to_numpy()
cnr = df['cnr'].to_numpy()
efc = df['efc'].to_numpy()
fber = df['fber'].to_numpy()
fwhm_avg = df['fwhm_avg'].to_numpy()
fwhm_x = df['fwhm_x'].to_numpy()
fwhm_y = df['fwhm_y'].to_numpy()
fwhm_z = df['fwhm_z'].to_numpy()
icvs_csf = df['icvs_csf'].to_numpy()
icvs_gm = df['icvs_gm'].to_numpy()
icvs_wm = df['icvs_wm'].to_numpy()
inu_med = df['inu_med'].to_numpy()
inu_range = df['inu_range'].to_numpy()
qi_1 = df['qi_1'].to_numpy()
qi_2 = df['qi_2'].to_numpy()
rpve_csf = df['rpve_csf'].to_numpy()
rpve_gm = df['rpve_gm'].to_numpy()
rpve_wm = df['rpve_wm'].to_numpy()
snr_csf = df['snr_csf'].to_numpy()
snr_gm = df['snr_gm'].to_numpy()
snr_total = df['snr_total'].to_numpy()
snr_wm = df['snr_wm'].to_numpy()
snrd_csf = df['snrd_csf'].to_numpy()
snrd_gm = df['snrd_gm'].to_numpy()
snrd_total = df['snrd_total'].to_numpy()
snrd_wm = df['snrd_wm'].to_numpy()
summary_bg_k = df['summary_bg_k'].to_numpy()
summary_bg_mad = df['summary_bg_mad'].to_numpy()
summary_bg_mean = df['summary_bg_mean'].to_numpy()
summary_bg_median = df['summary_bg_median'].to_numpy()
summary_bg_n = df['summary_bg_n'].to_numpy()
summary_bg_p05 = df['summary_bg_p05'].to_numpy()
summary_bg_p95 = df['summary_bg_p95'].to_numpy()
summary_bg_stdv = df['summary_bg_stdv'].to_numpy()
summary_csf_k = df['summary_csf_k'].to_numpy()
summary_csf_mad = df['summary_csf_mad'].to_numpy()
summary_csf_mean = df['summary_csf_mean'].to_numpy()
summary_csf_median = df['summary_csf_median'].to_numpy()
summary_csf_n = df['summary_csf_n'].to_numpy()
summary_csf_p05 = df['summary_csf_p05'].to_numpy()
summary_csf_p95 = df['summary_csf_p95'].to_numpy()
summary_csf_stdv = df['summary_csf_stdv'].to_numpy()
summary_gm_k = df['summary_gm_k'].to_numpy()
summary_gm_mad = df['summary_gm_mad'].to_numpy()
summary_gm_mean = df['summary_gm_mean'].to_numpy()
summary_gm_median = df['summary_gm_median'].to_numpy()
summary_gm_n = df['summary_gm_n'].to_numpy()
summary_gm_p05 = df['summary_gm_p05'].to_numpy()
summary_gm_p95 = df['summary_gm_p95'].to_numpy()
summary_gm_stdv = df['summary_gm_stdv'].to_numpy()
summary_wm_k = df['summary_wm_k'].to_numpy()
summary_wm_mad = df['summary_wm_mad'].to_numpy()
summary_wm_mean = df['summary_wm_mean'].to_numpy()
summary_wm_median = df['summary_wm_median'].to_numpy()
summary_wm_n = df['summary_wm_n'].to_numpy()
summary_wm_p05 = df['summary_wm_p05'].to_numpy()
summary_wm_p95 = df['summary_wm_p95'].to_numpy()
summary_wm_stdv = df['summary_wm_stdv'].to_numpy()
tpm_overlap_csf = df['tpm_overlap_csf'].to_numpy()
tpm_overlap_gm = df['tpm_overlap_gm'].to_numpy()
tpm_overlap_wm = df['tpm_overlap_wm'].to_numpy()
wm2max = df['wm2max'].to_numpy()

# add all the metrics above to a list
metrics = [meri, cjv, cnr, efc, fber, fwhm_avg, fwhm_x, fwhm_y, fwhm_z, icvs_csf, icvs_gm, icvs_wm, inu_med, inu_range, qi_1, qi_2, rpve_csf, rpve_gm, rpve_wm, snr_csf, snr_gm, snr_total, snr_wm, snrd_csf, snrd_gm, snrd_total, snrd_wm, summary_bg_k, summary_bg_mad, summary_bg_mean, summary_bg_median, summary_bg_n, summary_bg_p05, summary_bg_p95, summary_bg_stdv, summary_csf_k, summary_csf_mad, summary_csf_mean, summary_csf_median, summary_csf_n, summary_csf_p05, summary_csf_p95, summary_csf_stdv, summary_gm_k, summary_gm_mad, summary_gm_mean, summary_gm_median, summary_gm_n, summary_gm_p05, summary_gm_p95, summary_gm_stdv, summary_wm_k, summary_wm_mad, summary_wm_mean, summary_wm_median, summary_wm_n, summary_wm_p05, summary_wm_p95, summary_wm_stdv, tpm_overlap_csf, tpm_overlap_gm, tpm_overlap_wm, wm2max]
metrics_name = [i for i in df.columns if i not in ['subject', 'sub number', 'comments', 'Sex', 'Age', 'Height', 'Weight', 'BMI', 'axial_length']]

ALPHA = 0.05

# correlation between Meri and all the metrics
for i in range(len(metrics)):
    # R-squared measures the proportion of the variance in one variable that is predictable from the other variable.
    # p_value is a measure of the evidence against a null hypothesis. A small p-value indicates strong evidence against the null hypothesis, while a large p-value indicates weak evidence against the null hypothesis 
    r_squared, p_value = stats.pearsonr(meri, metrics[i])
    if p_value < ALPHA:
        print(f'{metrics_name[i]}: r^2 = {r_squared:.4}, p-value = {p_value:.4}')

# plot Meri vs. all the metrics
# for i in range(len(metrics)):
#     plt.figure(figsize=(10, 10))
#     plt.scatter(meri, metrics[i])
#     plt.xlabel('Meri')
#     plt.ylabel(metrics_name[i])
#     plt.show()


quality: r^2 = 1.0, p-value = 0.0
size_z: r^2 = 0.2271, p-value = 0.03894
summary_bg_k: r^2 = -0.2219, p-value = 0.04379
summary_gm_mean: r^2 = -0.3131, p-value = 0.003957
summary_gm_n: r^2 = -0.2609, p-value = 0.0172
summary_wm_k: r^2 = -0.2303, p-value = 0.03619
summary_wm_mean: r^2 = -0.2242, p-value = 0.04156


/home/jaimebarranco/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


CI95

In [12]:
# subjects outside the 95% confidence interval for qi_2
relevant_metrics = [summary_bg_k, summary_gm_mean, summary_gm_n, summary_wm_k, summary_wm_mean]
relevant_metrics_name = ['summary_bg_k', 'summary_gm_mean', 'summary_gm_n', 'summary_wm_k', 'summary_wm_mean']

for i in range(len(relevant_metrics)):
    print(f'\n{relevant_metrics_name[i]}')
    mean = np.mean(relevant_metrics[i])
    std = np.std(relevant_metrics[i])
    lower = mean - 2*std
    upper = mean + 2*std
    print(f'Lower: {lower}, Upper: {upper}')
    for j in range(len(relevant_metrics[i])):
        if relevant_metrics[i][j] < lower or relevant_metrics[i][j] > upper:
            print(f'{j+1:03} - {relevant_metrics[i][j]}')


summary_bg_k
Lower: -14.583927872351477, Upper: 37.4560750404094
037 - 40.6587548340778
039 - 116.978478093923
065 - 43.2225229839192

summary_gm_mean
Lower: 749.1098498409962, Upper: 787.1616628310477
010 - 787.876496823232
020 - 787.188546087105
028 - 746.896190266065
061 - 745.498012687434
062 - 746.329037996085

summary_gm_n
Lower: 6143.864737424969, Upper: 28151.605142093103
002 - 29983
003 - 32370
054 - 33012

summary_wm_k
Lower: 0.1620723086873081, Upper: 0.818487684432192
016 - 0.120161022991804
022 - 0.821862467532863
038 - 0.82305761471255
048 - 0.85217867265451
050 - 0.90011624279788
062 - 0.822211147938027
081 - 1.03999070604355

summary_wm_mean
Lower: 1001.9101787060507, Upper: 1006.4046900103632
036 - 1006.57452224029


## Brainmask - Bene

Metrics correlation with Bene's ratings

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# read an excel file in .xlsx format
df = pd.read_excel('/home/jaimebarranco/Desktop/scores.xlsx', sheet_name='mreyeqc_brainmask_bene')

# metrics
meri = df['rating'].to_numpy()
cjv = df['cjv'].to_numpy()
cnr = df['cnr'].to_numpy()
efc = df['efc'].to_numpy()
fber = df['fber'].to_numpy()
fwhm_avg = df['fwhm_avg'].to_numpy()
fwhm_x = df['fwhm_x'].to_numpy()
fwhm_y = df['fwhm_y'].to_numpy()
fwhm_z = df['fwhm_z'].to_numpy()
icvs_csf = df['icvs_csf'].to_numpy()
icvs_gm = df['icvs_gm'].to_numpy()
icvs_wm = df['icvs_wm'].to_numpy()
inu_med = df['inu_med'].to_numpy()
inu_range = df['inu_range'].to_numpy()
qi_1 = df['qi_1'].to_numpy()
qi_2 = df['qi_2'].to_numpy()
rpve_csf = df['rpve_csf'].to_numpy()
rpve_gm = df['rpve_gm'].to_numpy()
rpve_wm = df['rpve_wm'].to_numpy()
snr_csf = df['snr_csf'].to_numpy()
snr_gm = df['snr_gm'].to_numpy()
snr_total = df['snr_total'].to_numpy()
snr_wm = df['snr_wm'].to_numpy()
snrd_csf = df['snrd_csf'].to_numpy()
snrd_gm = df['snrd_gm'].to_numpy()
snrd_total = df['snrd_total'].to_numpy()
snrd_wm = df['snrd_wm'].to_numpy()
summary_bg_k = df['summary_bg_k'].to_numpy()
summary_bg_mad = df['summary_bg_mad'].to_numpy()
summary_bg_mean = df['summary_bg_mean'].to_numpy()
summary_bg_median = df['summary_bg_median'].to_numpy()
summary_bg_n = df['summary_bg_n'].to_numpy()
summary_bg_p05 = df['summary_bg_p05'].to_numpy()
summary_bg_p95 = df['summary_bg_p95'].to_numpy()
summary_bg_stdv = df['summary_bg_stdv'].to_numpy()
summary_csf_k = df['summary_csf_k'].to_numpy()
summary_csf_mad = df['summary_csf_mad'].to_numpy()
summary_csf_mean = df['summary_csf_mean'].to_numpy()
summary_csf_median = df['summary_csf_median'].to_numpy()
summary_csf_n = df['summary_csf_n'].to_numpy()
summary_csf_p05 = df['summary_csf_p05'].to_numpy()
summary_csf_p95 = df['summary_csf_p95'].to_numpy()
summary_csf_stdv = df['summary_csf_stdv'].to_numpy()
summary_gm_k = df['summary_gm_k'].to_numpy()
summary_gm_mad = df['summary_gm_mad'].to_numpy()
summary_gm_mean = df['summary_gm_mean'].to_numpy()
summary_gm_median = df['summary_gm_median'].to_numpy()
summary_gm_n = df['summary_gm_n'].to_numpy()
summary_gm_p05 = df['summary_gm_p05'].to_numpy()
summary_gm_p95 = df['summary_gm_p95'].to_numpy()
summary_gm_stdv = df['summary_gm_stdv'].to_numpy()
summary_wm_k = df['summary_wm_k'].to_numpy()
summary_wm_mad = df['summary_wm_mad'].to_numpy()
summary_wm_mean = df['summary_wm_mean'].to_numpy()
summary_wm_median = df['summary_wm_median'].to_numpy()
summary_wm_n = df['summary_wm_n'].to_numpy()
summary_wm_p05 = df['summary_wm_p05'].to_numpy()
summary_wm_p95 = df['summary_wm_p95'].to_numpy()
summary_wm_stdv = df['summary_wm_stdv'].to_numpy()
tpm_overlap_csf = df['tpm_overlap_csf'].to_numpy()
tpm_overlap_gm = df['tpm_overlap_gm'].to_numpy()
tpm_overlap_wm = df['tpm_overlap_wm'].to_numpy()
wm2max = df['wm2max'].to_numpy()

# add all the metrics above to a list
metrics = [meri, cjv, cnr, efc, fber, fwhm_avg, fwhm_x, fwhm_y, fwhm_z, icvs_csf, icvs_gm, icvs_wm, inu_med, inu_range, qi_1, qi_2, rpve_csf, rpve_gm, rpve_wm, snr_csf, snr_gm, snr_total, snr_wm, snrd_csf, snrd_gm, snrd_total, snrd_wm, summary_bg_k, summary_bg_mad, summary_bg_mean, summary_bg_median, summary_bg_n, summary_bg_p05, summary_bg_p95, summary_bg_stdv, summary_csf_k, summary_csf_mad, summary_csf_mean, summary_csf_median, summary_csf_n, summary_csf_p05, summary_csf_p95, summary_csf_stdv, summary_gm_k, summary_gm_mad, summary_gm_mean, summary_gm_median, summary_gm_n, summary_gm_p05, summary_gm_p95, summary_gm_stdv, summary_wm_k, summary_wm_mad, summary_wm_mean, summary_wm_median, summary_wm_n, summary_wm_p05, summary_wm_p95, summary_wm_stdv, tpm_overlap_csf, tpm_overlap_gm, tpm_overlap_wm, wm2max]
metrics_name = [i for i in df.columns if i not in ['subject', 'sub number', 'comments', 'Sex', 'Age', 'Height', 'Weight', 'BMI', 'axial_length']]

ALPHA = 0.05

# correlation between Meri and all the metrics
for i in range(len(metrics)):
    # R-squared measures the proportion of the variance in one variable that is predictable from the other variable.
    # p_value is a measure of the evidence against a null hypothesis. A small p-value indicates strong evidence against the null hypothesis, while a large p-value indicates weak evidence against the null hypothesis 
    r_squared, p_value = stats.pearsonr(meri, metrics[i])
    if p_value < ALPHA:
        print(f'{metrics_name[i]}: r^2 = {r_squared:.4}, p-value = {p_value:.4}')

# plot Meri vs. all the metrics
# for i in range(len(metrics)):
#     plt.figure(figsize=(10, 10))
#     plt.scatter(meri, metrics[i])
#     plt.xlabel('Meri')
#     plt.ylabel(metrics_name[i])
#     plt.show()


quality: r^2 = 1.0, p-value = 0.0
cnr: r^2 = 0.2242, p-value = 0.04161
summary_bg_median: r^2 = -0.2265, p-value = 0.03947
summary_gm_k: r^2 = -0.2206, p-value = 0.04507
summary_gm_mean: r^2 = -0.2832, p-value = 0.009473
summary_gm_n: r^2 = -0.2364, p-value = 0.03141


/home/jaimebarranco/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


CI95

In [14]:
# subjects outside the 95% confidence interval for qi_2
relevant_metrics = [cnr, summary_bg_median, summary_gm_k, summary_gm_mean, summary_gm_n]
relevant_metrics_name = ['cnr', 'summary_bg_median', 'summary_gm_k', 'summary_gm_mean', 'summary_gm_n']

for i in range(len(relevant_metrics)):
    print(f'\n{relevant_metrics_name[i]}')
    mean = np.mean(relevant_metrics[i])
    std = np.std(relevant_metrics[i])
    lower = mean - 2*std
    upper = mean + 2*std
    print(f'Lower: {lower}, Upper: {upper}')
    for j in range(len(relevant_metrics[i])):
        if relevant_metrics[i][j] < lower or relevant_metrics[i][j] > upper:
            print(f'{j+1:03} - {relevant_metrics[i][j]}')


cnr
Lower: 2.5256336293545703, Upper: 3.51481117653755
015 - 3.5193756625607
061 - 3.58777080768549

summary_bg_median
Lower: 0.8175354972981421, Upper: 7.086051898857489
003 - 0.0
029 - 0.0
034 - 0.0
037 - 0.0
047 - 0.0
050 - 0.0
051 - 0.0
065 - 0.0

summary_gm_k
Lower: -0.14862775506176742, Upper: 0.207331339712489
001 - 0.211930893862272
016 - 0.222340679813799
023 - 0.230171698554317
035 - 0.28360445106391
054 - -0.170411915886417

summary_gm_mean
Lower: 749.1098498409962, Upper: 787.1616628310477
010 - 787.876496823232
020 - 787.188546087105
028 - 746.896190266065
061 - 745.498012687434
062 - 746.329037996085

summary_gm_n
Lower: 6143.864737424969, Upper: 28151.605142093103
002 - 29983
003 - 32370
054 - 33012


## Eyemask - Meri

Metrics correlation

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# read an excel file in .xlsx format
df = pd.read_excel('/home/jaimebarranco/Desktop/scores.xlsx', sheet_name='mreyeqc_eyemask_meri')

# metrics
meri = df['rating'].to_numpy()
cjv = df['cjv'].to_numpy()
cnr = df['cnr'].to_numpy()
efc = df['efc'].to_numpy()
fber = df['fber'].to_numpy()
fwhm_avg = df['fwhm_avg'].to_numpy()
fwhm_x = df['fwhm_x'].to_numpy()
fwhm_y = df['fwhm_y'].to_numpy()
fwhm_z = df['fwhm_z'].to_numpy()
icvs_csf = df['icvs_csf'].to_numpy()
icvs_gm = df['icvs_gm'].to_numpy()
icvs_wm = df['icvs_wm'].to_numpy()
inu_med = df['inu_med'].to_numpy()
inu_range = df['inu_range'].to_numpy()
qi_1 = df['qi_1'].to_numpy()
qi_2 = df['qi_2'].to_numpy()
rpve_csf = df['rpve_csf'].to_numpy()
rpve_gm = df['rpve_gm'].to_numpy()
rpve_wm = df['rpve_wm'].to_numpy()
snr_csf = df['snr_csf'].to_numpy()
snr_gm = df['snr_gm'].to_numpy()
snr_total = df['snr_total'].to_numpy()
snr_wm = df['snr_wm'].to_numpy()
snrd_csf = df['snrd_csf'].to_numpy()
snrd_gm = df['snrd_gm'].to_numpy()
snrd_total = df['snrd_total'].to_numpy()
snrd_wm = df['snrd_wm'].to_numpy()
summary_bg_k = df['summary_bg_k'].to_numpy()
summary_bg_mad = df['summary_bg_mad'].to_numpy()
summary_bg_mean = df['summary_bg_mean'].to_numpy()
summary_bg_median = df['summary_bg_median'].to_numpy()
summary_bg_n = df['summary_bg_n'].to_numpy()
summary_bg_p05 = df['summary_bg_p05'].to_numpy()
summary_bg_p95 = df['summary_bg_p95'].to_numpy()
summary_bg_stdv = df['summary_bg_stdv'].to_numpy()
summary_csf_k = df['summary_csf_k'].to_numpy()
summary_csf_mad = df['summary_csf_mad'].to_numpy()
summary_csf_mean = df['summary_csf_mean'].to_numpy()
summary_csf_median = df['summary_csf_median'].to_numpy()
summary_csf_n = df['summary_csf_n'].to_numpy()
summary_csf_p05 = df['summary_csf_p05'].to_numpy()
summary_csf_p95 = df['summary_csf_p95'].to_numpy()
summary_csf_stdv = df['summary_csf_stdv'].to_numpy()
summary_gm_k = df['summary_gm_k'].to_numpy()
summary_gm_mad = df['summary_gm_mad'].to_numpy()
summary_gm_mean = df['summary_gm_mean'].to_numpy()
summary_gm_median = df['summary_gm_median'].to_numpy()
summary_gm_n = df['summary_gm_n'].to_numpy()
summary_gm_p05 = df['summary_gm_p05'].to_numpy()
summary_gm_p95 = df['summary_gm_p95'].to_numpy()
summary_gm_stdv = df['summary_gm_stdv'].to_numpy()
summary_wm_k = df['summary_wm_k'].to_numpy()
summary_wm_mad = df['summary_wm_mad'].to_numpy()
summary_wm_mean = df['summary_wm_mean'].to_numpy()
summary_wm_median = df['summary_wm_median'].to_numpy()
summary_wm_n = df['summary_wm_n'].to_numpy()
summary_wm_p05 = df['summary_wm_p05'].to_numpy()
summary_wm_p95 = df['summary_wm_p95'].to_numpy()
summary_wm_stdv = df['summary_wm_stdv'].to_numpy()
tpm_overlap_csf = df['tpm_overlap_csf'].to_numpy()
tpm_overlap_gm = df['tpm_overlap_gm'].to_numpy()
tpm_overlap_wm = df['tpm_overlap_wm'].to_numpy()
wm2max = df['wm2max'].to_numpy()

# add all the metrics above to a list
metrics = [meri, cjv, cnr, efc, fber, fwhm_avg, fwhm_x, fwhm_y, fwhm_z, icvs_csf, icvs_gm, icvs_wm, inu_med, inu_range, qi_1, qi_2, rpve_csf, rpve_gm, rpve_wm, snr_csf, snr_gm, snr_total, snr_wm, snrd_csf, snrd_gm, snrd_total, snrd_wm, summary_bg_k, summary_bg_mad, summary_bg_mean, summary_bg_median, summary_bg_n, summary_bg_p05, summary_bg_p95, summary_bg_stdv, summary_csf_k, summary_csf_mad, summary_csf_mean, summary_csf_median, summary_csf_n, summary_csf_p05, summary_csf_p95, summary_csf_stdv, summary_gm_k, summary_gm_mad, summary_gm_mean, summary_gm_median, summary_gm_n, summary_gm_p05, summary_gm_p95, summary_gm_stdv, summary_wm_k, summary_wm_mad, summary_wm_mean, summary_wm_median, summary_wm_n, summary_wm_p05, summary_wm_p95, summary_wm_stdv, tpm_overlap_csf, tpm_overlap_gm, tpm_overlap_wm, wm2max]
metrics_name = [i for i in df.columns if i not in ['subject', 'sub number', 'comments', 'Sex', 'Age', 'Height', 'Weight', 'BMI', 'axial_length']]

ALPHA = 0.05

# correlation between Meri and all the metrics
for i in range(len(metrics)):
    # R-squared measures the proportion of the variance in one variable that is predictable from the other variable.
    # p_value is a measure of the evidence against a null hypothesis. A small p-value indicates strong evidence against the null hypothesis, while a large p-value indicates weak evidence against the null hypothesis 
    r_squared, p_value = stats.pearsonr(meri, metrics[i])
    if p_value < ALPHA:
        print(f'{metrics_name[i]}: r^2 = {r_squared:.4}, p-value = {p_value:.4}')

# plot Meri vs. all the metrics
# for i in range(len(metrics)):
#     plt.figure(figsize=(10, 10))
#     plt.scatter(meri, metrics[i])
#     plt.xlabel('Meri')
#     plt.ylabel(metrics_name[i])
#     plt.show()


quality: r^2 = 1.0, p-value = 0.0
summary_csf_mean: r^2 = -0.2537, p-value = 0.02064


/home/jaimebarranco/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


CI95

In [16]:
# subjects outside the 95% confidence interval for qi_2
relevant_metrics = [summary_csf_mean]
relevant_metrics_name = ['summary_csf_mean']

for i in range(len(relevant_metrics)):
    print(f'\n{relevant_metrics_name[i]}')
    mean = np.mean(relevant_metrics[i])
    std = np.std(relevant_metrics[i])
    lower = mean - 2*std
    upper = mean + 2*std
    print(f'Lower: {lower}, Upper: {upper}')
    for j in range(len(relevant_metrics[i])):
        if relevant_metrics[i][j] < lower or relevant_metrics[i][j] > upper:
            print(f'{j+1:03} - {relevant_metrics[i][j]}')


summary_csf_mean
Lower: 383.270363659152, Upper: 518.2389633130906
006 - 545.550482497902
008 - 372.054525300511
031 - 378.030537036904
058 - 552.119631097792


## Eyemask - Bene

Metrics correlation

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# read an excel file in .xlsx format
df = pd.read_excel('/home/jaimebarranco/Desktop/scores.xlsx', sheet_name='mreyeqc_eyemask_bene')

# metrics
meri = df['rating'].to_numpy()
cjv = df['cjv'].to_numpy()
cnr = df['cnr'].to_numpy()
efc = df['efc'].to_numpy()
fber = df['fber'].to_numpy()
fwhm_avg = df['fwhm_avg'].to_numpy()
fwhm_x = df['fwhm_x'].to_numpy()
fwhm_y = df['fwhm_y'].to_numpy()
fwhm_z = df['fwhm_z'].to_numpy()
icvs_csf = df['icvs_csf'].to_numpy()
icvs_gm = df['icvs_gm'].to_numpy()
icvs_wm = df['icvs_wm'].to_numpy()
inu_med = df['inu_med'].to_numpy()
inu_range = df['inu_range'].to_numpy()
qi_1 = df['qi_1'].to_numpy()
qi_2 = df['qi_2'].to_numpy()
rpve_csf = df['rpve_csf'].to_numpy()
rpve_gm = df['rpve_gm'].to_numpy()
rpve_wm = df['rpve_wm'].to_numpy()
snr_csf = df['snr_csf'].to_numpy()
snr_gm = df['snr_gm'].to_numpy()
snr_total = df['snr_total'].to_numpy()
snr_wm = df['snr_wm'].to_numpy()
snrd_csf = df['snrd_csf'].to_numpy()
snrd_gm = df['snrd_gm'].to_numpy()
snrd_total = df['snrd_total'].to_numpy()
snrd_wm = df['snrd_wm'].to_numpy()
summary_bg_k = df['summary_bg_k'].to_numpy()
summary_bg_mad = df['summary_bg_mad'].to_numpy()
summary_bg_mean = df['summary_bg_mean'].to_numpy()
summary_bg_median = df['summary_bg_median'].to_numpy()
summary_bg_n = df['summary_bg_n'].to_numpy()
summary_bg_p05 = df['summary_bg_p05'].to_numpy()
summary_bg_p95 = df['summary_bg_p95'].to_numpy()
summary_bg_stdv = df['summary_bg_stdv'].to_numpy()
summary_csf_k = df['summary_csf_k'].to_numpy()
summary_csf_mad = df['summary_csf_mad'].to_numpy()
summary_csf_mean = df['summary_csf_mean'].to_numpy()
summary_csf_median = df['summary_csf_median'].to_numpy()
summary_csf_n = df['summary_csf_n'].to_numpy()
summary_csf_p05 = df['summary_csf_p05'].to_numpy()
summary_csf_p95 = df['summary_csf_p95'].to_numpy()
summary_csf_stdv = df['summary_csf_stdv'].to_numpy()
summary_gm_k = df['summary_gm_k'].to_numpy()
summary_gm_mad = df['summary_gm_mad'].to_numpy()
summary_gm_mean = df['summary_gm_mean'].to_numpy()
summary_gm_median = df['summary_gm_median'].to_numpy()
summary_gm_n = df['summary_gm_n'].to_numpy()
summary_gm_p05 = df['summary_gm_p05'].to_numpy()
summary_gm_p95 = df['summary_gm_p95'].to_numpy()
summary_gm_stdv = df['summary_gm_stdv'].to_numpy()
summary_wm_k = df['summary_wm_k'].to_numpy()
summary_wm_mad = df['summary_wm_mad'].to_numpy()
summary_wm_mean = df['summary_wm_mean'].to_numpy()
summary_wm_median = df['summary_wm_median'].to_numpy()
summary_wm_n = df['summary_wm_n'].to_numpy()
summary_wm_p05 = df['summary_wm_p05'].to_numpy()
summary_wm_p95 = df['summary_wm_p95'].to_numpy()
summary_wm_stdv = df['summary_wm_stdv'].to_numpy()
tpm_overlap_csf = df['tpm_overlap_csf'].to_numpy()
tpm_overlap_gm = df['tpm_overlap_gm'].to_numpy()
tpm_overlap_wm = df['tpm_overlap_wm'].to_numpy()
wm2max = df['wm2max'].to_numpy()

# add all the metrics above to a list
metrics = [meri, cjv, cnr, efc, fber, fwhm_avg, fwhm_x, fwhm_y, fwhm_z, icvs_csf, icvs_gm, icvs_wm, inu_med, inu_range, qi_1, qi_2, rpve_csf, rpve_gm, rpve_wm, snr_csf, snr_gm, snr_total, snr_wm, snrd_csf, snrd_gm, snrd_total, snrd_wm, summary_bg_k, summary_bg_mad, summary_bg_mean, summary_bg_median, summary_bg_n, summary_bg_p05, summary_bg_p95, summary_bg_stdv, summary_csf_k, summary_csf_mad, summary_csf_mean, summary_csf_median, summary_csf_n, summary_csf_p05, summary_csf_p95, summary_csf_stdv, summary_gm_k, summary_gm_mad, summary_gm_mean, summary_gm_median, summary_gm_n, summary_gm_p05, summary_gm_p95, summary_gm_stdv, summary_wm_k, summary_wm_mad, summary_wm_mean, summary_wm_median, summary_wm_n, summary_wm_p05, summary_wm_p95, summary_wm_stdv, tpm_overlap_csf, tpm_overlap_gm, tpm_overlap_wm, wm2max]
metrics_name = [i for i in df.columns if i not in ['subject', 'sub number', 'comments', 'Sex', 'Age', 'Height', 'Weight', 'BMI', 'axial_length']]

ALPHA = 0.05

# correlation between Meri and all the metrics
for i in range(len(metrics)):
    # R-squared measures the proportion of the variance in one variable that is predictable from the other variable.
    # p_value is a measure of the evidence against a null hypothesis. A small p-value indicates strong evidence against the null hypothesis, while a large p-value indicates weak evidence against the null hypothesis 
    r_squared, p_value = stats.pearsonr(meri, metrics[i])
    if p_value < ALPHA:
        print(f'{metrics_name[i]}: r^2 = {r_squared:.4}, p-value = {p_value:.4}')

# plot Meri vs. all the metrics
# for i in range(len(metrics)):
#     plt.figure(figsize=(10, 10))
#     plt.scatter(meri, metrics[i])
#     plt.xlabel('Meri')
#     plt.ylabel(metrics_name[i])
#     plt.show()


quality: r^2 = 1.0, p-value = 0.0


/home/jaimebarranco/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


CI95

In [18]:
# subjects outside the 95% confidence interval for qi_2
relevant_metrics = [summary_csf_mean]
relevant_metrics_name = ['summary_csf_mean']

for i in range(len(relevant_metrics)):
    print(f'\n{relevant_metrics_name[i]}')
    mean = np.mean(relevant_metrics[i])
    std = np.std(relevant_metrics[i])
    lower = mean - 2*std
    upper = mean + 2*std
    print(f'Lower: {lower}, Upper: {upper}')
    for j in range(len(relevant_metrics[i])):
        if relevant_metrics[i][j] < lower or relevant_metrics[i][j] > upper:
            print(f'{j+1:03} - {relevant_metrics[i][j]}')


summary_csf_mean
Lower: 383.270363659152, Upper: 518.2389633130906
006 - 545.550482497902
008 - 372.054525300511
031 - 378.030537036904
058 - 552.119631097792
